## Dependencies

In [1]:
import os
from dataclasses import dataclass
from pathlib import Path
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk,load_dataset

[2025-01-25 16:05:14,926: INFO: config: PyTorch version 2.5.1+cu118 available.]
[2025-01-25 16:05:14,929: INFO: config: TensorFlow version 2.18.0 available.]


In [2]:
%pwd

'c:\\Projects\\python\\text-summarizer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Projects\\python\\text-summarizer'

## Entity

In [5]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    TOKENIZER: Path

## Configuration Manager

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        # paths accessed via constant.py -> config.yaml,params.yaml
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root]) # due to configbox setup

    def get_data_transformation_config(self) -> DataTransformationConfig:

        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            TOKENIZER=config.TOKENIZER
        )

        return data_transformation_config

## Component

In [7]:
class DataTransformation:
    def __init__(self,config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=config.TOKENIZER)
    
    def extract_features(self,input_batch):
        input_encodings = self.tokenizer(input_batch['dialogue'],truncation=True,max_length=1024)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(input_batch['summary'],truncation=True,max_length=128)
        
        return {
            'input_ids':input_encodings['input_ids'],
            'attention_mask':input_encodings['attention_mask'],
            'labels':target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt= dataset_samsum.map(self.extract_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,'samsum_dataset'))

    

## Test

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-01-25 16:05:15,211: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-25 16:05:15,214: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-25 16:05:15,215: INFO: common: created directory at: artifacts]
[2025-01-25 16:05:15,216: INFO: common: created directory at: artifacts/data_transformation]


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Projects\python\mainenv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\harsh\.cache\huggingface\hub\models--sshleifer--distilbart-cnn-12-6. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Map:   0%|          | 0/2347 [00:00<?, ? examples/s]

Map:   0%|          | 0/2877 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25884 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2347 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2877 [00:00<?, ? examples/s]